In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
There is no [win32] chromedriver for browser 94.0.4606 in cache
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\TriciaToffey\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
dog_url = 'https://www.akc.org/most-popular-breeds/2017-full-list/'
browser.visit(dog_url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
breeds = soup.find_all('table')
breeds

[<table class="alignleft" style="width: 677px; height: 6870px;">
 <tbody>
 <tr style="height: 22px;">
 <td style="height: 22px; width: 587px;"><b>Breed</b></td>
 <td style="height: 22px; width: 74px;"><b>Rank</b></td>
 </tr>
 <tr style="height: 30px;">
 <td style="height: 30px; width: 587px;"><a href="/dog-breeds/labrador-retriever/"><span style="font-weight: 400;">Retrievers (Labrador)</span></a></td>
 <td style="height: 30px; width: 74px;"><span style="font-weight: 400;">1</span></td>
 </tr>
 <tr style="height: 30px;">
 <td style="height: 30px; width: 587px;"><a href="/dog-breeds/german-shepherd-dog/"><span style="font-weight: 400;">German Shepherd Dogs</span></a></td>
 <td style="height: 30px; width: 74px;"><span style="font-weight: 400;">2</span></td>
 </tr>
 <tr style="height: 30px;">
 <td style="height: 30px; width: 587px;"><a href="/dog-breeds/golden-retriever/"><span style="font-weight: 400;">Retrievers (Golden)</span></a></td>
 <td style="height: 30px; width: 74px;"><span styl

In [6]:
data_frame = pd.read_html(str(breeds))[0]
data_frame

,0,1
0,Breed,Rank
1,Retrievers (Labrador),1
2,German Shepherd Dogs,2
3,Retrievers (Golden),3
4,French Bulldogs,4
...,...,...
186,Otterhounds,186
187,American Foxhounds,187
188,Sloughis,188
189,English Foxhounds,189


In [7]:
dog_df = data_frame.rename(columns ={0:"Breed",1:"breed_rank"})
dog_df

,Breed,breed_rank
0,Breed,Rank
1,Retrievers (Labrador),1
2,German Shepherd Dogs,2
3,Retrievers (Golden),3
4,French Bulldogs,4
...,...,...
186,Otterhounds,186
187,American Foxhounds,187
188,Sloughis,188
189,English Foxhounds,189


In [8]:
dog_df = dog_df.iloc[1:]
dog_df

,Breed,breed_rank
1,Retrievers (Labrador),1
2,German Shepherd Dogs,2
3,Retrievers (Golden),3
4,French Bulldogs,4
5,Bulldogs,5
...,...,...
186,Otterhounds,186
187,American Foxhounds,187
188,Sloughis,188
189,English Foxhounds,189


In [9]:
max(dog_df['Breed'].value_counts().to_list())

1

In [10]:
dog_df.reset_index(inplace = True)

In [11]:
dog_df

,index,Breed,breed_rank
0,1,Retrievers (Labrador),1
1,2,German Shepherd Dogs,2
2,3,Retrievers (Golden),3
3,4,French Bulldogs,4
4,5,Bulldogs,5
...,...,...,...
185,186,Otterhounds,186
186,187,American Foxhounds,187
187,188,Sloughis,188
188,189,English Foxhounds,189


In [12]:
dog_df=dog_df.rename(columns = {'index':'breed_id'})

In [13]:
dog_df

,breed_id,Breed,breed_rank
0,1,Retrievers (Labrador),1
1,2,German Shepherd Dogs,2
2,3,Retrievers (Golden),3
3,4,French Bulldogs,4
4,5,Bulldogs,5
...,...,...,...
185,186,Otterhounds,186
186,187,American Foxhounds,187
187,188,Sloughis,188
188,189,English Foxhounds,189


In [14]:
new_index = dog_df.set_index(['breed_id'])

In [15]:
new_index

,Breed,breed_rank
breed_id,,
1,Retrievers (Labrador),1
2,German Shepherd Dogs,2
3,Retrievers (Golden),3
4,French Bulldogs,4
5,Bulldogs,5
...,...,...
186,Otterhounds,186
187,American Foxhounds,187
188,Sloughis,188


In [16]:
new_index.dtypes

Breed         object
breed_rank    object
dtype: object

In [17]:
new_index[['breed_rank']]=new_index[['breed_rank']].apply(pd.to_numeric)

In [18]:
dog_index = new_index

In [19]:
dog_index.to_csv("dog_index.csv")

In [20]:
characteristics_df = pd.read_csv('characteristics_df.csv')

In [21]:
characteristics_df = characteristics_df.drop(labels = 'Unnamed: 0', axis=1)

In [24]:
from sqlalchemy import create_engine

#from sql_pass import username, password

In [26]:
#rds_connection_string = f"{username}:{password}@localhost:5432/dog_new_db"
rds_connection_string = "ttoffey:postgres@localhost:5432/dog_new_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [36]:
engine.table_names()

[]

In [37]:
new_index.to_sql(name='dog_breed', con=engine, if_exists='append', index=False)

In [38]:
pd.read_sql_query('select * from dog_breed', con=engine).head()

,Breed,breed_rank
0,Retrievers (Labrador),1
1,German Shepherd Dogs,2
2,Retrievers (Golden),3
3,French Bulldogs,4
4,Bulldogs,5


In [39]:
characteristics_df.to_sql(name='dog_character', con=engine, if_exists='append', index=False)

NameError: name 'characteristics_df' is not defined

In [40]:
pd.read_sql_query('select * from dog_character', con=engine).head()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "dog_character" does not exist
LINE 1: select * from dog_character
                      ^

[SQL: select * from dog_character]
(Background on this error at: http://sqlalche.me/e/f405)

In [33]:
pd.read_sql_query('select * from dog_character where "Average_Price" = 1000.0', con=engine).head()

,BreedName,show_groups,subcategories,weight,Temperament,Average_Price,Intelligence,watchdog,breed_rank
0,Affenpinscher,Toy,Companion,4.0,"Active, Adventurous, Curious, Fun-loving, Play...",1000.0,37.0,1.0,147
1,Akita,Working,Herding,44.0,"Alert, Courageous, Dignified, Docile, Friendly...",1000.0,54.0,5.0,47
2,Bloodhound,Hound,Scenthound,46.0,"Affectionate, Even-tempered, Gentle, Stubborn,...",1000.0,74.0,1.0,50
3,Clumber Spaniel,Sporting,Gun,32.0,"Gentle, Loyal, Affectionate, Dignified, Aloof",1000.0,37.0,3.0,140
4,Great Dane,Working,Guardian,77.0,"Friendly, Confident, Devoted, Intelligent, Gen...",1000.0,48.0,5.0,14
